In [ ]:

import requests
import pandas as pd
from datetime import datetime, timedelta

# Function to download onion prices
def download_onion_prices():
    try:
        # Get today's date and last week's date
        today = datetime.now()
        last_week = today - timedelta(days=7)
        
        # Format the dates in the required format for the website
        today_date = today.strftime('%d/%m/%Y')
        last_week_date = last_week.strftime('%d/%m/%Y')
        
        # Define the URL to download data
        url = f'https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=23&Tx_State=MH&Tx_District=13&Tx_Market=169&DateFrom=01-Mar-2023&DateTo=12-Apr-2023&Fr_Date=01-Mar-2023&To_Date=12-Apr-2023&Tx_Trend=0&Tx_CommodityHead=Onion&Tx_StateHead=Maharashtra&Tx_DistrictHead=Nashik&Tx_MarketHead=Lasalgaon'
        
        # Send a GET request to the website
        response = requests.get(url)
        
        # Check if response is successful
        if response.status_code == 200:
            # Extract data from the response
            data = pd.read_html(response.content)
            
            # Get the relevant data from the extracted data
            df = data[0]
            
            # Load previously downloaded data, if available
            try:
                old_data = pd.read_excel('onion_prices.xlsx')
                # Concatenate new data with old data
                df = pd.concat([old_data, df], ignore_index=True)
            except FileNotFoundError:
                pass
            
            # Save the data to an Excel file
            filename = f'onion_prices.xlsx'
            df.to_excel(filename, index=False)
            
            print(f'Successfully downloaded onion prices to {filename}')
        else:
            print('Failed to download onion prices. Please check if the website is accessible.')
    except Exception as e:
        print(f'Error occurred: {str(e)}')

# Call the function to download onion prices
download_onion_prices()


Successfully downloaded onion prices to onion_prices.xlsx


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

url = 'https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=23&Tx_State=MH&Tx_District=13&Tx_Market=169&DateFrom=01-Mar-2023&DateTo=12-Apr-2023&Fr_Date=01-Mar-2023&To_Date=12-Apr-2023&Tx_Trend=0&Tx_CommodityHead=Onion&Tx_StateHead=Maharashtra&Tx_DistrictHead=Nashik&Tx_MarketHead=Lasalgaon'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'tableagmark_new'})
rows = table.find_all('tr')

filename = 'onion_pricesbing.csv'
with open(filename, mode='a', newline='') as file:
    writer = csv.writer(file)
    for row in rows[1:]:
        cols = row.find_all('td')
        date_str = cols[0].text.strip()
        price_str = cols[2].text.strip()
        date_obj = datetime.strptime(date_str, '%d-%b-%Y')
        writer.writerow([date_obj, price_str])

print(f'Successfully downloaded and saved onion prices to {filename}!')

ValueError: ignored

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from dateutil import parser
from datetime import datetime, timedelta

# Set date range from today to past one year
today = datetime.now().date()
one_year_ago = today - timedelta(days=365)

# Fetch HTML page
url = "https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=23&Tx_State=MH&Tx_District=13&Tx_Market=169&DateFrom=01-Mar-2023&DateTo=12-Apr-2023&Fr_Date=01-Mar-2023&To_Date=12-Apr-2023&Tx_Trend=0&Tx_CommodityHead=Onion&Tx_StateHead=Maharashtra&Tx_DistrictHead=Nashik&Tx_MarketHead=Lasalgaon"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract relevant information from the HTML
prices = []
dates = []
for row in soup.find_all("tr"):
    cols = row.find_all("td")
    if len(cols) == 10:  # Assumes the relevant data is in rows with 8 columns
        date_str = cols[9].get_text(strip=True)
        price = cols[8].get_text(strip=True)
        date = parser.parse(date_str).date()
        if date >= one_year_ago and date <= today:
            prices.append(price)
            dates.append(date)

# Create a dataframe
df = pd.DataFrame({"Date": dates, "Price": prices})

# Load old data from CSV file
try:
    old_df = pd.read_csv("onion_prices.csv")
except FileNotFoundError:
    old_df = pd.DataFrame(columns=["Date", "Price"])

# Concatenate old and new data
merged_df = pd.concat([old_df, df], ignore_index=True)

# Store data in CSV file
merged_df.to_csv("onion_prices.csv", index=False)

print("Onion prices have been downloaded and stored in onion_prices.csv")


Onion prices have been downloaded and stored in onion_prices.csv


In [ ]:
#13-04-23
import os
import csv
import datetime
import requests
from bs4 import BeautifulSoup

# Define the URL and headers
url = 'https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=23&Tx_State=MH&Tx_District=13&Tx_Market=169&DateFrom=01-Apr-2023&DateTo=13-Apr-2023&Fr_Date=01-Apr-2023&To_Date=13-Apr-2023&Tx_Trend=0&Tx_CommodityHead=Onion&Tx_StateHead=Maharashtra&Tx_DistrictHead=Nashik&Tx_MarketHead=Lasalgaon'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

# Get today's date and date range for last week to today
today = datetime.date.today()
last_week = today - datetime.timedelta(days=7)

# Format dates for URL
today_str = today.strftime('%d/%m/%Y')
last_week_str = last_week.strftime('%d/%m/%Y')

# Make GET request to the website
response = requests.get(url.format(last_week_str, today_str), headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the data
table = soup.find('table', {'class': 'tableagmark_new'})

# Check if table is found
if table:
    # Create a CSV file to store the data
    csv_file_path = os.path.join('onion_prices11.csv')
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        
        # Write headers to the CSV file
        headers = [th.text for th in table.findAll('th')]
        writer.writerow(headers)
        
        # Write rows to the CSV file
        rows = table.findAll('tr')
        for row in rows[1:]:
            data = [td.text for td in row.findAll('td')]
            writer.writerow(data)
        
    print('Data has been downloaded and saved to onion_prices.csv.')
else:
    print('Table not found. Please check the URL or website structure.')


Data has been downloaded and saved to onion_prices.csv.


In [ ]:
#13-04-2023
import os
import csv
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import requests
from bs4 import BeautifulSoup

# Define the URL and headers
url = 'https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=23&Tx_State=MH&Tx_District=13&Tx_Market=169&DateFrom=01-Apr-2023&DateTo=13-Apr-2023&Fr_Date=01-Apr-2023&To_Date=13-Apr-2023&Tx_Trend=0&Tx_CommodityHead=Onion&Tx_StateHead=Maharashtra&Tx_DistrictHead=Nashik&Tx_MarketHead=Lasalgaon'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

# Get today's date and date range for last week to today
today = parse('2023-04-13').date()
last_week = today - relativedelta(days=7)

# Format dates for URL
today_str = today.strftime('%d/%m/%Y')
last_week_str = last_week.strftime('%d/%m/%Y')

# Make GET request to the website
response = requests.get(url.format(last_week_str, today_str), headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the data
table = soup.find('table', {'class': 'tableagmark_new'})

# Check if table is found
if table:
    # Create a CSV file to store the data
    csv_file_path = os.path.join('onion_prices.csv')
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)

        # Write headers to the CSV file
        headers = [th.text for th in table.findAll('th')]
        writer.writerow(headers)

        # Write rows to the CSV file
        rows = table.findAll('tr')
        for row in rows[1:]:
            data = [td.text for td in row.findAll('td')]
            writer.writerow(data)

    print('Data has been downloaded and saved to onion_prices.csv.')
else:
    print('Table not found. Please check the URL or website structure.')


Data has been downloaded and saved to onion_prices.csv.


In [ ]:
#new file 13-04-23

import os
import csv
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
import requests
from bs4 import BeautifulSoup

# Define the URL and headers
url = 'https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=23&Tx_State=MH&Tx_District=13&Tx_Market=169&DateFrom=01-Apr-2023&DateTo=13-Apr-2023&Fr_Date=01-Apr-2023&To_Date=13-Apr-2023&Tx_Trend=0&Tx_CommodityHead=Onion&Tx_StateHead=Maharashtra&Tx_DistrictHead=Nashik&Tx_MarketHead=Lasalgaon'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

# Get today's date and date range for last week to today
today = parse('2023-04-13').date()
last_week = today - relativedelta(days=7)

# Format dates for URL
today_str = today.strftime('%d/%m/%Y')
last_week_str = last_week.strftime('%d/%m/%Y')

# Make GET request to the website
response = requests.get(url.format(last_week_str, today_str), headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the data
table = soup.find('table', {'class': 'tableagmark_new'})

# Check if table is found
if table:
    # Create a CSV file to store the data
    csv_file_path = os.path.join('onion_prices_test.csv')
    # Check if the CSV file already exists
    if not os.path.exists(csv_file_path):
        # Write headers to the CSV file if it doesn't exist
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)
            headers = [th.text for th in table.findAll('th')]
            writer.writerow(headers)

    # Append new rows to the existing CSV file
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        # Write rows to the CSV file
        rows = table.findAll('tr')
        for row in rows[1:]:
            data = [td.text for td in row.findAll('td')]
            writer.writerow(data)

    print('Data has been downloaded and appended to onion_prices.csv.')
else:
    print('Table not found. Please check the URL or website structure.')


Data has been downloaded and appended to onion_prices.csv.


In [ ]:
#vscode file
import os
import csv
import requests
from bs4 import BeautifulSoup
from datetime import date, timedelta

# Define the URL and headers
url = 'https://www.commodityinsightsx.com/commodities/mandi-prices/onion-market-price-in-lasalgaon'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'}

# Get today's date and date range for last week to today
today = date.today()
last_week = today - timedelta(days=7)

# Format dates for URL
today_str = today.strftime('%d/%m/%Y')
last_week_str = last_week.strftime('%d/%m/%Y')

# Make GET request to the website
response = requests.get(url.format(last_week_str, today_str, today_str, today_str), headers=headers)

# Parse the HTML content
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing the data
table = soup.find('table', {'class': 'tableagmark_new'})

# Check if table is found
if table:
    # Create a CSV file to store the data
    csv_file_path = os.path.join('onion_prices_vscode.csv')
    # Check if the CSV file already exists
    if not os.path.exists(csv_file_path):
        # Write headers to the CSV file if it doesn't exist
        with open(csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
            writer = csv.writer(csv_file)
            headers = [th.text for th in table.findAll('th')]
            writer.writerow(headers)

    # Append new rows to the existing CSV file
    with open(csv_file_path, 'a', newline='', encoding='utf-8') as csv_file:
        writer = csv.writer(csv_file)
        # Write rows to the CSV file
        rows = table.findAll('tr')
        for row in rows[1:]:
            data = [td.text for td in row.findAll('td')]
            writer.writerow(data)

    print('Data has been downloaded and appended to onion_prices.csv.')
else:
    print('Table not found. Please check the URL or website structure.')




Table not found. Please check the URL or website structure.


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import datetime

# Function to fetch onion prices data from the website
def fetch_onion_prices():
    # Define URL and headers
    url = 'https://www.commodityonline.com/market-place/spot/lasalgaon/40/32'
    headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36'
    }

    # Fetch web page
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract onion prices data from the web page
    rows = soup.find_all('tr', {'class': 'row-card'})
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if cols:
            data.append(cols)
    return data

# Function to write onion prices data to CSV file
def write_to_csv(data):
    filename = 'onion_prices.csv'
    with open(filename, 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

# Fetch onion prices data
onion_prices_data = fetch_onion_prices()

# Write onion prices data to CSV file
write_to_csv(onion_prices_data)

# Get today's date
now = datetime.datetime.now()
today = now.strftime("%Y-%m-%d")

# Fetch and write onion prices data for each day in the last week up to today
for i in range(1, 8):
    date = (now - datetime.timedelta(days=i)).strftime("%Y-%m-%d")
    url = f'https://www.commodityinsightsx.com/commodities/mandi-prices/onion-market-price-in-lasalgaon'
    # response = requests.get(url, headers=headers)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    rows = soup.find_all('tr', {'class': 'row-card'})
    data = []
    for row in rows:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        if cols:
            data.append(cols)
    write_to_csv(data)

print(f'Successfully downloaded onion prices data from {today} to {date} and appended to onion_prices.csv.')


Successfully downloaded onion prices data from 2023-04-18 to 2023-04-11 and appended to onion_prices.csv.


In [ ]:
import pandas as pd
df = pd.read_csv("/content/onion_prices_vscode.csv")

In [ ]:
df

Sl no. District Name    Market Name  Commodity    Variety    Grade  \
0        1    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
1        2    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
2        3    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
3        4    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
4        5    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
5        6    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
6        7    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
7        8    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
8        9    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
9       10    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
10      11    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
11      12    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
12      13    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
13      14    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   
14      15    \nNashik\n  \nLasalgaon\n  \nOnion\n    \nRed\n  \nFAQ\n   
15      16    \nNashik\n  \nLasalgaon\n  \nOnion\n  \nOther\n  \nFAQ\n   

    Min Price (Rs./Quintal)  Max Price (Rs./Quintal)  \
0                       300                      778   
1                       400                     1111   
2                       200                      681   
3                       400                     1045   
4                       201                      706   
5                       400                      981   
6                       300                      716   
7                       500                     1056   
8                       301                      868   
9                       400                     1090   
10                      300                      751   
11                      600                     1090   
12                      300                      700   
13                      551                     1080   
14                      200                      653   
15                      500                     1091   

    Modal Price (Rs./Quintal)       Price Date  
0                         620  \n13 Apr 2023\n  
1                         750  \n13 Apr 2023\n  
2                         500  \n12 Apr 2023\n  
3                         700  \n12 Apr 2023\n  
4                         600  \n11 Apr 2023\n  
5                         760  \n11 Apr 2023\n  
6                         551  \n10 Apr 2023\n  
7                         800  \n10 Apr 2023\n  
8                         570  \n08 Apr 2023\n  
9                         800  \n08 Apr 2023\n  
10                        600  \n07 Apr 2023\n  
11                        870  \n07 Apr 2023\n  
12                        600  \n06 Apr 2023\n  
13                        825  \n06 Apr 2023\n  
14                        500  \n05 Apr 2023\n  
15                        801  \n05 Apr 2023\n

In [ ]:
#new websiteee
import requests
from bs4 import BeautifulSoup
import csv
import datetime

# Specify the URL
url = "https://agriplus.in/price/onion/maharashtra/nashik/lasalgaon"

# Send GET request to the website
response = requests.get(url)

# Parse the HTML content
soup = BeautifulSoup(response.content, "html.parser")

# Find the table containing the data
table = soup.find("table")

# Create a CSV file to store the data
csv_file = open("onion_prices.csv", "a")
csv_writer = csv.writer(csv_file)

# Find all rows in the table
rows = table.findAll("tr")

# Loop through each row and extract the data
for row in rows:
    cells = row.findAll("td")
    if len(cells) > 0:
        date = cells[0].text.strip()
        variety = cells[1].text.strip()
        min_price = cells[2].text.strip()
        max_price = cells[3].text.strip()
        modal_price = cells[4].text.strip()

        # Write the data to the CSV file
        csv_writer.writerow([date, variety, min_price, max_price, modal_price])

# Close the CSV file
csv_file.close()

# Print a message indicating that the data has been successfully scraped and saved
print("Data has been scraped and saved to onion_prices.csv")


Data has been scraped and saved to onion_prices.csv


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv

# Define the URL to scrape
url = "https://www.commodityinsightsx.com/commodities/mandi-prices/onion-market-price-in-lasalgaon"

# Define the month to scrape prices for
month = "April"

# Function to scrape and append onion prices to CSV file
def scrape_and_append_onion_prices():
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Extract the table with prices
    table = soup.find('table', class_='table-bordered')

    # Extract the headers from the table
    headers = [header.text.strip() for header in table.findAll('th')]

    # Extract the rows for the given month from the table
    rows = []
    for row in table.findAll('tr')[1:]:
        cells = [cell.text.strip() for cell in row.findAll('td')]
        if cells[0].startswith(month):
            rows.append(cells)

    # Write the rows to CSV file
    with open('prices.csv', 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(rows)

# Call the scraping function
scrape_and_append_onion_prices()



In [ ]:
#new webiste vs code
import requests
from bs4 import BeautifulSoup
import csv
import datetime

# Function to scrape and append onion prices
def scrape_and_append_prices():
    # Specify the URL
    url = "https://agriplus.in/price/onion/maharashtra/nashik/lasalgaon"

    # Send GET request to the website
    response = requests.get(url)

    # Parse the HTML content
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the table containing the data
    table = soup.find("table")

    # Find all rows in the table
    rows = table.findAll("tr")

    # Get today's date
    today = datetime.datetime.now().date()

    # Open the CSV file in append mode
    with open("prices.csv", "a", newline='') as csv_file:
        csv_writer = csv.writer(csv_file)

        # Loop through each row and extract the data
        for row in rows:
            cells = row.findAll("td")
            if len(cells) > 0:
                date = cells[0].text.strip()
                variety = cells[1].text.strip()
                min_price = cells[2].text.strip()
                max_price = cells[3].text.strip()
                modal_price = cells[4].text.strip()

                # Append the data to the CSV file
                csv_writer.writerow([today, date, variety, min_price, max_price, modal_price])

    # Print a message indicating that the data has been successfully scraped and appended
    print("Data has been scraped and appended to onion_prices.csv")

# Call the function to scrape and append onion prices
scrape_and_append_prices()

Data has been scraped and appended to onion_prices.csv


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import os

# URL of the webpage to scrape
url = 'https://agriplus.in/price/onion/maharashtra/nashik/lasalgaon'

# Send a GET request to the URL
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find('table')  # Find the table element containing the data

    # Extract table headers
    headers = [th.text.strip() for th in table.findAll('th')]

    # Extract table rows
    rows = []
    for row in table.findAll('tr'):
        row_data = [td.text.strip() for td in row.findAll('td')]
        if row_data:  # Skip empty rows
            rows.append(row_data)

    # Convert data to a pandas DataFrame
    df = pd.DataFrame(rows, columns=headers)

    # Generate a filename for the CSV file using current date
    now = datetime.datetime.now()
    filename = f'onionprices.csv'

    # Check if the file already exists
    if os.path.exists(filename):
        # Append data to the existing CSV file
        df.to_csv(filename, mode='a', header=False, index=False)
        print(f'Data appended to {filename} successfully.')
    else:
        # Create a new CSV file and write data to it
        df.to_csv(filename, mode='w', header=True, index=False)
        print(f'New file {filename} created and data written successfully.')
else:
    print('Failed to download webpage')


New file onionprices.csv created and data written successfully.


In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import os.path
import datetime

def get_data():
    url = "https://agriplus.in/price/onion/maharashtra/nashik/lasalgaon"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')
    data = []
    for row in rows[1:]:
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols]
        data.append(cols)
    return data

def append_data(file_path, data):
    file_exists = os.path.isfile(file_path)
    with open(file_path, 'a') as f:
        writer = csv.writer(f)
        if not file_exists:
            writer.writerow(['Price Date', 'Variety', 'Min Price (Rs./Quintal)', 'Max Price (Rs./Quintal)', 'Modal Price (Rs./Quintal)'])
        writer.writerows(data)

def main():
    today = datetime.date.today().strftime("%d-%m-%Y")
    file_path = 'onions.csv'
    data = get_data()
    existing_data = []
    if os.path.isfile(file_path):
        with open(file_path, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                if today in row:
                    print('Data already exists for today')
                    return
                existing_data.append(row)
    data_to_append = [row for row in data if row not in existing_data]
    if data_to_append:
        append_data(file_path, data_to_append)
        print(f'Successfully appended {len(data_to_append)} rows to {file_path}')
    else:
        print('No new data to append')

if __name__ == '__main__':
    main()


No new data to append
